In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [2]:
spark = SparkSession.builder.appName('session-07').getOrCreate()
spark

In [21]:
df = spark.read.csv('../data/tips.csv',
                    header=True,
                    inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [22]:
df.dtypes

[('total_bill', 'double'),
 ('tip', 'double'),
 ('sex', 'string'),
 ('smoker', 'string'),
 ('day', 'string'),
 ('time', 'string'),
 ('size', 'int')]

In [23]:
df.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [24]:
indexer = StringIndexer(
    inputCols=['sex', 'smoker', 'day', 'time'],
    outputCols=['sex_indexed', 'smoker_indexed', 'day_indexed', 'time_indexed']
)

df_processed = indexer.fit(df).transform(df)
df_processed.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [25]:
feature_assembler = VectorAssembler(
    inputCols=['tip', 'sex_indexed', 'smoker_indexed', 'day_indexed', 'time_indexed', 'size'],
    outputCol='feature_vector'
)

df_processed = feature_assembler.transform(df_processed)

In [26]:
final_df = df_processed.select('feature_vector', 'total_bill')
final_df.show()

+--------------------+----------+
|      feature_vector|total_bill|
+--------------------+----------+
|[1.01,1.0,0.0,1.0...|     16.99|
|[1.66,0.0,0.0,1.0...|     10.34|
|[3.5,0.0,0.0,1.0,...|     21.01|
|[3.31,0.0,0.0,1.0...|     23.68|
|[3.61,1.0,0.0,1.0...|     24.59|
|[4.71,0.0,0.0,1.0...|     25.29|
|[2.0,0.0,0.0,1.0,...|      8.77|
|[3.12,0.0,0.0,1.0...|     26.88|
|[1.96,0.0,0.0,1.0...|     15.04|
|[3.23,0.0,0.0,1.0...|     14.78|
|[1.71,0.0,0.0,1.0...|     10.27|
|[5.0,1.0,0.0,1.0,...|     35.26|
|[1.57,0.0,0.0,1.0...|     15.42|
|[3.0,0.0,0.0,1.0,...|     18.43|
|[3.02,1.0,0.0,1.0...|     14.83|
|[3.92,0.0,0.0,1.0...|     21.58|
|[1.67,1.0,0.0,1.0...|     10.33|
|[3.71,0.0,0.0,1.0...|     16.29|
|[3.5,1.0,0.0,1.0,...|     16.97|
|(6,[0,5],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [27]:
train_data, test_data = final_df.randomSplit([0.25, 0.75])
model = LinearRegression(featuresCol='feature_vector',
                         labelCol='total_bill')
model = model.fit(train_data)

In [28]:
result_summary = model.evaluate(test_data)

In [30]:
result_summary.predictions.show()

+--------------------+----------+------------------+
|      feature_vector|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,5],[1.25,2.0])|     10.07| 9.078786573580397|
|(6,[0,5],[1.25,2.0])|     10.51| 9.078786573580397|
|(6,[0,5],[1.97,2.0])|     12.02|11.956527829332627|
| (6,[0,5],[2.0,2.0])|     12.69| 12.07643371498897|
| (6,[0,5],[2.0,2.0])|     13.37| 12.07643371498897|
|(6,[0,5],[2.01,2.0])|     20.23|12.116402343541083|
|(6,[0,5],[2.24,3.0])|     16.04| 16.39911381366096|
|(6,[0,5],[2.34,4.0])|     17.81|20.162233112603346|
| (6,[0,5],[2.5,4.0])|     18.35|20.801731169437176|
|(6,[0,5],[2.64,3.0])|     17.59|17.997858955745528|
|(6,[0,5],[2.72,2.0])|     13.28|14.954174970741201|
| (6,[0,5],[3.0,2.0])|      14.0|16.073296570200398|
| (6,[0,5],[3.0,4.0])|     20.45|22.800162597042885|
|(6,[0,5],[3.15,3.0])|     20.08| 20.03625901190336|
|(6,[0,5],[3.18,2.0])|     19.82|16.792731884138462|
|(6,[0,5],[3.27,2.0])|     17.78|17.1524495411

In [31]:
result_summary.meanAbsoluteError, result_summary.meanSquaredError

(4.535212582830957, 42.08169257414803)